# Estrutura de Dados - Blockchain Cartório

* nome do pai é não-obrigatório
* não registrar data de nascimento se houver data de óbito
* não registrar casamento se tiver data de óbito
* não registrar separação se tiver data de óbito
* somente registrar separação se houver casamento e não houver separação
* registrar viuvez somente se houver registro de obito de um dos conjuges
* não restringir casamento de mesmo sexo

O Hash deve ser gerado para o conteúdo todo do bloco.

Deve ser implementada um lista encadeada cujo ID seja esse HASH.

Somente a operação de inserção deve ser realizada, respeitando as regras de negócio.

In [1]:
from datetime import datetime
from hashlib import sha256

class Pessoa():
  def __init__(self, cpf, nome, sexo, data_nascimento, nome_mae, nome_pai = None):
    self.cpf = cpf
    self.nome = nome
    self.sexo = sexo
    self.nome_pai = nome_pai
    self.nome_mae = nome_mae
    self.data_nascimento = data_nascimento
    self.data_obito = None
    self.em_casamento = False
    self.cpf_pessoa_casada = None

  def set_data_obito(self, data_obito):
    self.data_obito = data_obito

  def set_casamento(self, cpf_pessoa_casada):
    self.em_casamento = True
    self.cpf_pessoa_casada = cpf_pessoa_casada

  def set_separacao(self):
    self.em_casamento = False
    self.cpf_pessoa_casada = None

  def set_viuvez(self):
    self.em_casamento = False
    self.cpf_pessoa_casada = None

class ListaPessoa():
  def __init__(self):
    self.lista = []

  def adicionar_pessoa(self, pessoa):
    self.lista.append(pessoa)

class Registro():
  def __init__(self, transactions, data, hash_anterior):
    self.transactions = transactions
    self.data = data
    self.hash_anterior = hash_anterior
    self.nonce = 0
    self.timestamp = datetime.now()
    self.hash = self.gerar_hash()

  def imprimir_registro(self):
    print("Timestamp: ", self.timestamp)
    print("Transactions: ", self.transactions)
    print("Data: ", self.data)
    print("Hash: ", self.gerar_hash())
    print("Hash Anterior: ", self.hash_anterior)
    print("\n")

  def gerar_hash(self):
    conteudo = str(self.timestamp) + str(self.transactions) + str(self.data) + str(self.hash_anterior) + str(self.nonce)
    block_hash = sha256(conteudo.encode())
    return block_hash.hexdigest()

class ListaRegistro():
  def __init__(self):
    self.lista = []
    self.all_transactions = []
    self.iniciar_lista()
    
  def iniciar_lista(self):
    transactions = []
    previous_hash = "0"
    data = []
    self.lista.append(Registro(transactions, data, previous_hash))

  def imprimir_lista_registro(self):
    for i in range(len(self.lista)):
      registro = self.lista[i]
      print("Block {} {}".format(i, registro))
      print(registro.hash)
      registro.imprimir_registro()

  def adicionar_registro(self, transactions, data):
    hash_registro_anterior = self.lista[len(self.lista)-1].hash
    novo_registro = Registro(transactions, data, hash_registro_anterior)
    self.lista.append(novo_registro)

class Cartorio():
  def __init__(self):
    self.lista_pessoa = ListaPessoa()
    self.lista_registro = ListaRegistro()

  def registrar_nascimento(self, pessoa):
    for i in range(len(self.lista_pessoa.lista)):
        if i > 0 and self.lista_pessoa.lista[i].cpf == pessoa.cpf:
          print("Já existe uma pessoa com o CPF inserido!")
          return None
    self.lista_pessoa.adicionar_pessoa(pessoa)
    data = {"cpf":pessoa.cpf,"nome":pessoa.nome,"sexo":pessoa.sexo,"nome_pai":pessoa.nome_pai,"nome_mae":pessoa.nome_mae,"data_nascimento":pessoa.data_nascimento}
    self.lista_registro.adicionar_registro('Nascimento', data)
    print(f"Registro de Nascimento para {pessoa.cpf} executado")

  def registrar_obito(self, pessoa, data_obito):
    registro_executado, casamento = False, False
    for i in range(len(self.lista_pessoa.lista)):
        if len(self.lista_pessoa.lista) > 0 and self.lista_pessoa.lista[i].cpf == pessoa.cpf:
          if self.lista_pessoa.lista[i].data_obito != None:
            print("Já existe um registro de óbito para esta pessoa")
            return None
          else:
            self.lista_pessoa.lista[i].set_data_obito(data_obito)
            registro_executado = True
            if self.lista_pessoa.lista[i].em_casamento == True:
              casamento = True
              conjugue_falecido = self.lista_pessoa.lista[i]
              cpf_conjugue = self.lista_pessoa.lista[i].cpf_pessoa_casada
              self.lista_pessoa.lista[i].set_separacao()
    if registro_executado == False:
      pessoa.set_data_obito(data_obito)
      self.lista_pessoa.adicionar_pessoa(pessoa)
    data = {"cpf":pessoa.cpf,"nome":pessoa.nome,"sexo":pessoa.sexo,"nome_pai":pessoa.nome_pai,"nome_mae":pessoa.nome_mae,"data_nascimento":pessoa.data_nascimento,"data_obito":pessoa.data_obito}
    self.lista_registro.adicionar_registro('Óbito', data)
    print(f"Registro de Óbito para {pessoa.cpf} executado")
    if casamento == True:
      self.registrar_viuvez(cpf_conjugue, conjugue_falecido)

  def registrar_casamento(self, conjugue1, conjugue2):
    conjugue1_cadastrado, conjugue2_cadastrado = False, False

    for i in range(len(self.lista_pessoa.lista)):
      if self.lista_pessoa.lista[i].cpf == conjugue1.cpf or self.lista_pessoa.lista[i].cpf == conjugue2.cpf:
        if self.lista_pessoa.lista[i].cpf == conjugue1.cpf:
          conjugue1_cadastrado = True
          i_conjugue1 = i
          if  self.lista_pessoa.lista[i].data_obito != None or self.lista_pessoa.lista[i].em_casamento == True:
            print("Casamento inválido!")
            return None
        else:
          conjugue2_cadastrado = True
          i_conjugue2 = i
          if  self.lista_pessoa.lista[i].data_obito != None or self.lista_pessoa.lista[i].em_casamento == True:
            print("Casamento inválido!")
            return None
    if conjugue1_cadastrado == False:
      conjugue1.set_casamento(conjugue2.cpf)
      self.lista_pessoa.adicionar_pessoa(conjugue1)
    else:
      self.lista_pessoa.lista[i_conjugue1].set_casamento(conjugue2.cpf)
    if conjugue2_cadastrado == False:
      conjugue2.set_casamento(conjugue1.cpf)
      self.lista_pessoa.adicionar_pessoa(conjugue2)
    else:
      self.lista_pessoa.lista[i_conjugue2].set_casamento(conjugue1.cpf)
    data = [
            {"Conjugue01":conjugue1.nome,"cpf":conjugue1.cpf,"sexo":conjugue1.sexo,"nome_pai":conjugue1.nome_pai,"nome_mae":conjugue1.nome_mae,"data_nascimento":conjugue1.data_nascimento},
            {"Conjugue02":conjugue2.nome,"cpf":conjugue2.cpf,"sexo":conjugue2.sexo,"nome_pai":conjugue2.nome_pai,"nome_mae":conjugue2.nome_mae,"data_nascimento":conjugue2.data_nascimento}
    ]
    self.lista_registro.adicionar_registro('Casamento', data)
    print(f"Registro de Casamento para {conjugue1.cpf} e {conjugue2.cpf} executado")

  def registrar_separacao(self, conjugue1, conjugue2):
    conjugue1_cadastrado, conjugue2_cadastrado = False, False
    for i in range(len(self.lista_pessoa.lista)):
      if self.lista_pessoa.lista[i].cpf == conjugue1.cpf or self.lista_pessoa.lista[i].cpf == conjugue2.cpf:
        if self.lista_pessoa.lista[i].cpf == conjugue1.cpf:
          conjugue1_cadastrado = True
          i_conjugue1 = i
          if self.lista_pessoa.lista[i].data_obito != None or self.lista_pessoa.lista[i].em_casamento == False or self.lista_pessoa.lista[i].cpf_pessoa_casada != conjugue2.cpf:
            print("Separação inválida")
            return None
        else:
          conjugue2_cadastrado = True
          i_conjugue2 = i
          if self.lista_pessoa.lista[i].data_obito != None or self.lista_pessoa.lista[i].em_casamento == False or self.lista_pessoa.lista[i].cpf_pessoa_casada != conjugue1.cpf:
            print("Separação inválida")
            return None
    if conjugue1_cadastrado == False:
      conjugue1.set_separacao()
      self.lista_pessoa.adicionar_pessoa(conjugue1)
    else:
      self.lista_pessoa.lista[i_conjugue1].set_separacao()
    if conjugue2_cadastrado == False:
      conjugue2.set_separacao()
      self.lista_pessoa.adicionar_pessoa(conjugue2)
    else:
      self.lista_pessoa.lista[i_conjugue2].set_separacao()
    data = [
            {"Conjugue01":conjugue1.nome,"cpf":conjugue1.cpf,"sexo":conjugue1.sexo,"nome_pai":conjugue1.nome_pai,"nome_mae":conjugue1.nome_mae,"data_nascimento":conjugue1.data_nascimento},
            {"Conjugue02":conjugue2.nome,"cpf":conjugue2.cpf,"sexo":conjugue2.sexo,"nome_pai":conjugue2.nome_pai,"nome_mae":conjugue2.nome_mae,"data_nascimento":conjugue2.data_nascimento}
    ]
    self.lista_registro.adicionar_registro('Separação', data)
    print(f"Registro de Separação para {conjugue1.cpf} e {conjugue2.cpf} executado")

  def registrar_viuvez(self, cpf, conjugue_falecido):
    for i in range(len(self.lista_pessoa.lista)):
      if self.lista_pessoa.lista[i].cpf == cpf:
        pessoa = self.lista_pessoa.lista[i]
        self.lista_pessoa.lista[i].set_viuvez()
    data = [
            {"Conjugue01":pessoa.nome,"cpf":pessoa.cpf,"sexo":pessoa.sexo,"nome_pai":pessoa.nome_pai,"nome_mae":pessoa.nome_mae,"data_nascimento":pessoa.data_nascimento},
            {"Conjugue02":conjugue_falecido.nome,"cpf":conjugue_falecido.cpf,"sexo":conjugue_falecido.sexo,"nome_pai":conjugue_falecido.nome_pai,"nome_mae":conjugue_falecido.nome_mae,"data_nascimento":conjugue_falecido.data_nascimento}
    ]
    self.lista_registro.adicionar_registro('Viuvez', data)
    print(f"Registro de Viuvez para {pessoa.cpf} executado")

################################################################################################
joao = Pessoa("111.111.111-10", "João", "masculino", "2021-12-25", "Maria", "Jose")
joana = Pessoa("111.111.111-11", "Joana", "feminino", "2021-12-25", "Joanilda", "Josefino")
jose = Pessoa("111.111.111-09", "Jose", "masculino", "2021-12-25", "Matilde", "Humberto")
maria = Pessoa("111.111.111-08", "Maria", "feminino", "2021-12-25", "Filomena", "Abel")

cartorio = Cartorio()

cartorio.registrar_nascimento(maria)
cartorio.registrar_nascimento(jose)
cartorio.registrar_nascimento(joao)
cartorio.registrar_nascimento(joana)

cartorio.registrar_casamento(jose, maria)
cartorio.registrar_casamento(joao, joana)

cartorio.registrar_obito(maria, "2021-12-25")

cartorio.registrar_separacao(joao, joana) 

cartorio.lista_registro.imprimir_lista_registro()

Registro de Nascimento para 111.111.111-08 executado
Registro de Nascimento para 111.111.111-09 executado
Registro de Nascimento para 111.111.111-10 executado
Registro de Nascimento para 111.111.111-11 executado
Registro de Casamento para 111.111.111-09 e 111.111.111-08 executado
Registro de Casamento para 111.111.111-10 e 111.111.111-11 executado
Registro de Óbito para 111.111.111-08 executado
Registro de Viuvez para 111.111.111-09 executado
Registro de Separação para 111.111.111-10 e 111.111.111-11 executado
Block 0 <__main__.Registro object at 0x7f08a4100730>
8254084910b9577fa3578887564428f1ea181e1e0783becaea2cd6bba42b5772
Timestamp:  2023-04-06 19:49:41.626107
Transactions:  []
Data:  []
Hash:  8254084910b9577fa3578887564428f1ea181e1e0783becaea2cd6bba42b5772
Hash Anterior:  0


Block 1 <__main__.Registro object at 0x7f08a7c18a60>
bce9c12af913f9b03efb8e3ea94e557ef5b954a9c35d6be5afef8e19c4b285ec
Timestamp:  2023-04-06 19:49:41.626218
Transactions:  Nascimento
Data:  {'cpf': '111.111.